# Eine kleine Einführung in Partielle Differentialgleichungen und die Finite Elemente Methode


Wir wollen die Poisson-Gleichung lösen:

Gegeben ist eine offene Menge $D \subset {\mathbb R}^2$ und eine Funktion $f : D \rightarrow {\mathbb R}$. 

Gesucht ist die Funktion $u : D \rightarrow {\mathbb R}$ sodass

$$
\Delta u (x,y) = f(x,y) \quad \forall \, (x,y) \in D,
$$

wobei der Laplace-Operator definiert ist als

$$
\Delta u (x,y) = 
\frac{\partial^2 u}{\partial x^2}(x,y) + \frac{\partial^2 u}{\partial y^2}(x,y)
$$

Die Lösung ist eindeutig wenn noch Randbedingungen gefordert werden:

$$
u(x,y) = 0 \quad \forall \, (x,y) \in \partial D
$$

Mit solchen **Partiellen Differentialgleichungen** können viele Aufgabenstellungen in den Naturwissenschaften, in technischen Anwendungen, als auch in der Finanzmathematik modelliert werden. Eine exakte Lösung zu finden ist oft nicht möglich, und man berechnet mit Computereinsatz näherungsweise Lösungen (Numerik).

## Approximation von Funktionen auf Triangulierungen

Wir verwenden dazu das Python - Paket NGSolve (www.ngsolve.org).

Wir wollen zuerst ein paar einfache numerische Operationen (differenzieren, integrieren) mit Funktionen durchführen:

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))

Wir verwenden die Funktion (ein *Gaussian-peak*):

$$
f(x,y) = \exp (-100 \, (  (x-\tfrac{1}{2})^2 + (y-\tfrac{1}{2})^2 )
$$

In [ ]:
func = exp(-100* ((x-0.5)**2+(y-0.5)**2))
ea = (-60,0,-1)
Draw (func, mesh, deformation=True, scale=0.3, euler_angles=ea);

Diese Funktion wird jetzt auf dem Finite-Elemente Netz **interpoliert**: Wir berechnen dazu die Funktionswerte in den Knoten, und ersetzen auf jedem einzelnen Dreieck die komplizierte Funktion durch eine affin-lineare Funktion der Bauart $a + b x + cy$, mit $a,b,c \in {\mathbb R}$, und mit den gleichen Funktionswerten in den Knoten.

In [ ]:
fes = H1(mesh, order=1)
gfu = GridFunction(fes)
gfu.Interpolate (func)
Draw (gfu, deformation=True, scale=0.3, euler_angles=ea);

## Berechnung von Integralen:

Wir wollen nun das Integral 

$$
\int_D f(x,y) \, d(x,y)
$$

berechnen. Das ist das Volumen unter dem Graphen der Funktion. 

Wir nutzen die interpolierte Funktion um das Integral näherungsweise (numerisch) zu berechnen: Wir ersetzen die Funktion durch das arithmetische Mittel der Werte in den Eckpunkten, berechnen das Volumen dieses Prismas nach Grundfläche $\times$ Höhe, und summieren über alle Dreiecke:

$$
\sum_{\text{Dreiecke}\, T}  \text{Fläche(T)} \sum_{\text{Eckpunkte}\, x_i} \frac{f(x_i)}{3}
$$

In [ ]:
Integrate (gfu, mesh)

Um die Genauigkeit zu erhöhen können wir ein feineres Netz verwenden. Eine effizientere Alternative ist es nicht mit affin-linearen, sondern mit Polynomen höherer Ordnung zu interpolieren:

In [ ]:
fes = H1(mesh, order=3, dirichlet=".*")
gfu = GridFunction(fes)
gfu.Interpolate (func)
Draw (gfu, deformation=True, scale=0.3, euler_angles=ea);

In [ ]:
Integrate (gfu, mesh)

Der Fehler zum exakten Wert $\pi/100$ beträgt:

In [ ]:
Integrate (gfu, mesh) - pi/100

## Berechnung von partiellen Ableitungen:

Wir wollen die partiellen Ableitungen

$$
\frac{\partial f}{\partial x} (x,y) = \lim_{\varepsilon \rightarrow 0} \frac{f(x+\varepsilon,y) - f(x,y) }{\varepsilon}
$$
und
$$
\frac{\partial f}{\partial y} (x,y) = \lim_{\varepsilon \rightarrow 0} \frac{f(x, y+\varepsilon) - f(x,y) }{\varepsilon}
$$

berechnen. Das sind die Steigungen der Funktion in $x$- bzw. in $y$-Richtung. Beide partiellen Ableitungen als Vektor geschrieben wird als **Gradient** bezeichnet:

$$
\nabla f := \operatorname{grad} \, f := \left( \frac{ \partial f}{\partial x}, \frac{\partial f}{\partial y} \right)
$$

In [ ]:
print ("df/dx:")
Draw (grad(gfu)[0], mesh)
print ("df/dy:")
Draw (grad(gfu)[1], mesh);

Und gleich der Gradient als Vektor gezeichnet, dargestellt durch Kegel: Die Spitze zeigt in in Richtung des steilsten Anstieges. Die Farbe entspricht der Norm des Gradienten, $\| \nabla f \|$, das ist die Steigung:

In [ ]:
Draw (grad(gfu), mesh, vectors={"grid_size":50});

# Projektion auf Teilräume

Wir wollen jetzt die gegebene Funktion nicht durch Interpolation annähern, sondern durch orthogonale Projektion bzgl. eines Skalarproduktes.

Sei $(V, \left<.,.\right>)$ ein Euklidscher Vektorraum, und $V_h \subset V$ ein linearer Teilraum. Für $u \in V$ ist die orthogonale Projektion $Pu \in V_h$ auf den Teilraum definiert sodass:

$$
\left< u - Pu, v \right> = 0\quad \forall \, v_h \in V_h
$$

Der Projektionsfehler $u - Pu$ steht orthogonal auf den Teilraum selbst. Diese Projektionsaufgabe können wir auch schreiben als:

$$
\text{Ges.:} \; u_h \in V_h : \left< u_h, v_h \right> = \left< u, v_h \right> \quad \forall \, v_h \in V_h
$$

Das Skalarprodukt auf der linken Seite ist bilinear, d.h. linear in beiden Argumenten. Darum wird es auch *Bilinearform* genannt:

$$
A : V \times V \rightarrow {\mathbb R} : \quad A(u,v) = \left< u, v \right>
$$

Für ein gegebenes, festes $u$ ist der Term auf der rechten Seite linear in $v$, also eine *Linearform*:

$$
f : V \rightarrow {\mathbb R} : \quad f(v) = \left< u, v \right>
$$

Das Problem lautet nun:

$$
\text{Ges.:} \; u_h \in V_h : A(u_h, v_h) = f(v_h) \quad \forall \, v_h \in V_h
$$


Wir verwenden für die Projektion das $L_2$ Skalarprodukt:

$$
\left< u, v \right>_{L_2(D)} := \int_D u(x,y) \, v(x,y) \, d(x,y)
$$

Dieses Skalarprodukt kann in NGSolve als `u*v*dx` angegeben werden. Die Projektion wird durch Aufstellen (Assemblierung) und Lösen eines linearen Gleichungssystems berechnet:

In [ ]:
fes = H1(mesh, order=2)
u,v = fes.TnT()

A = BilinearForm(u*v*dx).Assemble().mat
f = LinearForm(func*v*dx).Assemble().vec

gfu = GridFunction(fes)
gfu.vec.data = A.Inverse() * f

In [ ]:
Draw (gfu, deformation=True, scale=0.3, euler_angles=ea);

Die berechnete Funktion minimiert den Fehler in der $L_2$-Norm:

$$
\| u - u_h \|_{L_2} = \min_{v_h \in V_h} \| u - v_h \|_{L_2}
$$

In [ ]:
sqrt(sqrt (Integrate ( (gfu-func)**2, mesh)))

Die Berechnung der Projektion funktioniert so:

Wir wählen eine Basis $p_i$ für den Vektorraum $V_h$. Da $V_h$ ein Vektorraum von Funktionen ist, sind diese Basiselemente ebenfalls Funktionen $p_i(x,y)$. Im einfachsten Fall wählt man Funktionen die in einem Knoten 1 sind, in allen anderen 0, und dazwischen linear interpoliert:

In [ ]:
first = 200
num = 20
gf = GridFunction(fes, multidim=num)
for i in range(num):
    gf.vecs[i][:] = 0
    gf.vecs[i][first+i] = 1
scene = Draw (gf, mesh, deformation=True, animate=True, scale=0.1, euler_angles=[-60,5,60]);

Wir stellen die gesucht Lösung in dieser Basis dar (mit $N = \dim V_h$):

$$
u_h(x,y) = \sum_{i=1}^{N} u_i {\bf p}_i(x,y)
$$

Der Koeffizientenvektor $u = (u_1, \ldots u_n) \in {{\mathbb R}^N}$ wird durch die Projektionseigenschaft bestimmt: 

$$
A( \sum u_i {\bf p}_i, {\bf p}_j ) = f( {\bf p}_j ) \quad \forall \, j = 1, \ldots N
$$

Wir nutzen die Linearität von $A(.,.)$

$$
\sum_{i=1}^N A({\bf p}_i, {\bf p}_j ) \; u_i = f( {\bf p}_j ) \quad \forall \, j = 1, \ldots N,
$$

und erhalten das lineare Gleichungssystem für den Koeffizientenvektor $u$

$$
A u = f,
$$

mit den Einträgen der Matrix A und des rechte-Seite Vektors f:

\begin{eqnarray*}
A_{j,i} & = & A(p_i, p_j) = \int p_i p_j \, d(x,y) \\
f_j & = & f(p_j) = \int u \, p_j \, d(x,y)
\end{eqnarray*}


## Projektion mit dem $H^1$-Skalarprodukt

Wir kehren zurück zur Poisson-Gleichung:

Gesucht ist die Funktion $u(x,y)$ sodass:

$$
\Delta u(x,y) = f(x,y) \quad \forall \, (x,y) \in D
$$

und 

$$
u(x,y) = 0 \qquad \forall \, (x,y) \in \partial D
$$

Wir wollen die Projektion $u_h = P u$ der unbekannten Funktion $u$ auf den endlich-dimensionalen Raum $V_h$ berechnen. Als Skalarprodukt wählen wir jetzt (der Raum $H^1$ heißt Sobolev-Raum und ist ein Hilbert-Raum):

$$
\left< u, v \right>_{H^1} = \int_D \nabla u \cdot \nabla v \; d(x,y),
$$

wobei die Notation $\nabla u \cdot \nabla v = \left< \nabla u, \nabla v\right>_{{\mathbb R}^2}$ verwendet wird.
Analog zu oben bestimmen wir die Matrix $A$ mit Einträgen

$$
A_{ij} = \int_D \nabla p_i \cdot \nabla p_j \, d(x,y)
$$

und den Vektor $f$ mit Einträgen

$$
f_j = \int_D \nabla u \cdot \nabla p_j \, d(x,y) 
$$

Die Pointe ist dass dieser Vektor $f$ OHNE die unbekannte Funktion berechnet werden kann. Dazu nutzen wir die Green'sche Formel, ähnlich zur partiellen Integration:

$$
\int_D \nabla u \cdot \nabla p_j \, d(x,y) =
-\int_D \underbrace{\Delta u}_{f} \, p_j \, d(x,y) + \int_{\partial D} \nabla u \cdot \nu \, p_j 
$$

Hier steht nun die gegebene Funktion $f = \Delta u$, die Terme am Rand $\partial D$ verschwinden wegen der Randbedingung $u = 0$. 

In [ ]:
fes = H1(mesh, order=3, dirichlet=".*")
u,v = fes.TnT()

A = BilinearForm(grad(u)*grad(v)*dx).Assemble().mat
f = LinearForm(100*func*v*dx).Assemble().vec

gfu = GridFunction(fes)
gfu.vec.data = A.Inverse(fes.FreeDofs()) * f

Draw(gfu, deformation=True, scale=0.5, euler_angles=ea);

Zur Kontrolle berechnen wir $-\Delta u_h$:

Wir verwenden dazu die Hesse-Matrix
$$
\left( \begin{array}{cc}
      \frac{\partial^2 u}{\partial x^2}  & \frac{\partial^2 u}{\partial x \partial y}  \\
      \frac{\partial^2 u}{\partial x \partial y}  & \frac{\partial^2 u}{\partial y^2}
    \end{array} \right)
$$
und die Spur (engl: trace) einer Matrix als Summer der Diagonaleinträge:

In [ ]:
Draw (-Trace(gfu.Operator("hesse")), mesh, scale=0.003, deformation=True, euler_angles=ea);

# Ein paar weitere Differentialgleichungen:
* [Elastizität](elasticity3D.ipynb) beschreibt die Deformation elastischer Obejekte
* [Navier-Stokes Gleichung](navierstokes.ipynb) beschreiben die Strömung inkompressibler Flüssigkeiten
* [Maxwell'schen Gleichungen](coil.ipynb) beschreiben die Interaktion von elektrischen und magnetischen Feldern